In [21]:
from bs4 import BeautifulSoup
import requests

### Task 1: Get all links to the each event

In [30]:
base_url = 'http://www.ufcstats.com./statistics/events/completed'
all_urls = [base_url + f"?page={i}" for i in range(2,23)]
all_urls.insert(0,base_url)
all_urls

['http://www.ufcstats.com./statistics/events/completed',
 'http://www.ufcstats.com./statistics/events/completed?page=2',
 'http://www.ufcstats.com./statistics/events/completed?page=3',
 'http://www.ufcstats.com./statistics/events/completed?page=4',
 'http://www.ufcstats.com./statistics/events/completed?page=5',
 'http://www.ufcstats.com./statistics/events/completed?page=6',
 'http://www.ufcstats.com./statistics/events/completed?page=7',
 'http://www.ufcstats.com./statistics/events/completed?page=8',
 'http://www.ufcstats.com./statistics/events/completed?page=9',
 'http://www.ufcstats.com./statistics/events/completed?page=10',
 'http://www.ufcstats.com./statistics/events/completed?page=11',
 'http://www.ufcstats.com./statistics/events/completed?page=12',
 'http://www.ufcstats.com./statistics/events/completed?page=13',
 'http://www.ufcstats.com./statistics/events/completed?page=14',
 'http://www.ufcstats.com./statistics/events/completed?page=15',
 'http://www.ufcstats.com./statistics/eve

In [31]:
def get_links_event(urls):
    """ Return each link to the UFC events"""
    links = []
    for url in urls:
        r = requests.get(url)
        soup = BeautifulSoup(r.content)
        info_rows = soup.find_all('tr', class_='b-statistics__table-row')[2:]
    
        for row in info_rows:
            link = row.find('a')['href']
            links.append(link)
    return links
even_links = get_links_event(all_urls)
even_links[:10]

['http://www.ufcstats.com/event-details/d57e6a8971b6d2bd',
 'http://www.ufcstats.com/event-details/27541033b97c076d',
 'http://www.ufcstats.com/event-details/4a35913bd9aa4161',
 'http://www.ufcstats.com/event-details/9114c8ded5ccd71d',
 'http://www.ufcstats.com/event-details/6597b611f1c32555',
 'http://www.ufcstats.com/event-details/c36e1f4fa755ffb4',
 'http://www.ufcstats.com/event-details/8c90c1563972e44d',
 'http://www.ufcstats.com/event-details/6e2b1d631832921d',
 'http://www.ufcstats.com/event-details/81b57acd6975ac06',
 'http://www.ufcstats.com/event-details/bfe95ec546692b13']

### Task 2: Get information from each event

In [32]:
def get_links_fight(urls):
    """Return each link to the fight detail"""
    links = []
    for url in urls:
        r = requests.get(url)
        soup = BeautifulSoup(r.content)
        table = soup.find('table', class_='b-fight-details__table')
        rows = table.find_all('tr')
        for row in rows[1:]:
            links.append(row['data-link'])
    return links
fight_links = get_links_fight(even_links)
fight_links[:10]

['http://www.ufcstats.com/fight-details/394802bb096b403e',
 'http://www.ufcstats.com/fight-details/2f82e8079ba354f4',
 'http://www.ufcstats.com/fight-details/ec8d93ae99a7799d',
 'http://www.ufcstats.com/fight-details/ffd8e8575a551875',
 'http://www.ufcstats.com/fight-details/26202a4bd36d3fd2',
 'http://www.ufcstats.com/fight-details/c9c79de21823d941',
 'http://www.ufcstats.com/fight-details/d72f9b3ac742a9e3',
 'http://www.ufcstats.com/fight-details/2ef09c8fe29251ac',
 'http://www.ufcstats.com/fight-details/d6b5a6ad27a730ae',
 'http://www.ufcstats.com/fight-details/78dc032441394331']

### Task 3: Create dictionary of data for each fight

In [33]:
def get_info_box(soup_obj):
    """Return a dictinary about the information box (method, round,time,referee,and detail)"""
    my_dict = {}
    
    # get info box
    fight_title = soup_obj.find('div', class_='b-fight-details__fight-head').get_text(strip=True)
    my_dict['Fight Type'] = fight_title
    box = soup_obj.find('p', class_='b-fight-details__text')
    method = box.find_all(class_='b-fight-details__text-item_first')
    method = method[0].get_text('\n', strip=True).split('\n')
    if len(method) >=2:
        my_dict[method[0]] = method[1]
    
    # get the rest of content of info box 
    content = box.find_all(class_='b-fight-details__text-item')
    for item in content:
        pair = item.get_text('\n', strip=True).split('\n')
        if len(pair) >= 2:
            my_dict[pair[0]] = pair[1]
        else:
            my_dict[pair[0]] = None
    detail = box.find_next_sibling("p").get_text(strip=True).split(':')
    my_dict[detail[0]] = detail[1]
    
    return my_dict

def get_fight_detail(soup_obj):
    """Return a dictionary about fight details (fighter's names, significant strike"""
    my_dict = {}
    table = soup_obj.find('tbody', class_='b-fight-details__table-body')
    info = table.select('.b-fight-details__table-col')
    index = [0,2,3,4]
    info = [info[i].get_text('\n',strip=True) for i in index]
    
    # Red and blue figters
    fighters = info[0].split('\n')
    my_dict['r_figher'] = fighters[0]
    my_dict['b_figher'] = fighters[1]

    # significant strike
    sig_str = info[1].split('\n')
    my_dict['r_figher_sig_str'] = sig_str[0]
    my_dict['b_figher_sig_str'] = sig_str[1]

    # significant strik percentage
    sig_str_pct = info[2].split('\n')
    my_dict['r_figher_sig_str %'] = sig_str_pct[0]
    my_dict['b_figher_sig_str %'] = sig_str_pct[1]

    # total strike
    total_str = info[3].split('\n')
    my_dict['r_figher_total_str'] = total_str[0]
    my_dict['b_figher_total_str'] = total_str[1]

    return my_dict

def merge(dict1,dict2):
    """Merge two dictionaries"""
    res = {**dict1, **dict2}
    return res

def get_fight_info(url):
    """Return dictionary with information about the fight"""
    r = requests.get(url)
    soup = BeautifulSoup(r.content)
    
    # get info box
    dict_list = get_info_box(soup)
    # get the winner
    winner = soup.find('i', class_='b-fight-details__person-status_style_green')
    if winner is not None:
        winner = winner.find_next('div', class_='b-fight-details__person-text')
        winner = winner.find('a').get_text(strip=True)
        dict_list['Winner'] = winner
    else:
        dict_list['Winner'] = 'draw'

    # get fight info
    fight_info = get_fight_detail(soup)
    new_dict = merge(dict_list, fight_info)
    return new_dict

In [34]:
fight_info_list = []
for index, link in enumerate(fight_links):
    fight_info = get_fight_info(link)
    print("===================" + str(index) + "=====================")
    print(fight_info)
    fight_info_list.append(fight_info)

hod:': 'Submission', 'Round:': '3', 'Time:': '0:28', 'Time format:': '5 Rnd (5-5-5-5-5)', 'Referee:': 'Mike Beltran', 'Details': 'Guillotine Choke After Drop to Half Guard', 'Winner': 'Charles Oliveira', 'r_figher': 'Kevin Lee', 'b_figher': 'Charles Oliveira', 'r_figher_sig_str': '41 of 80', 'b_figher_sig_str': '43 of 65', 'r_figher_sig_str %': '51%', 'b_figher_sig_str %': '66%', 'r_figher_total_str': '61 of 100', 'b_figher_total_str': '51 of 73'}
===================535=====================
{'Fight Type': 'Welterweight Bout', 'Method:': 'KO/TKO', 'Round:': '1', 'Time:': '2:34', 'Time format:': '3 Rnd (5-5-5)', 'Referee:': 'Osiris Maia', 'Details': 'Punch to Head At Distance', 'Winner': 'Gilbert Burns', 'r_figher': 'Demian Maia', 'b_figher': 'Gilbert Burns', 'r_figher_sig_str': '4 of 7', 'b_figher_sig_str': '13 of 16', 'r_figher_sig_str %': '57%', 'b_figher_sig_str %': '81%', 'r_figher_total_str': '4 of 7', 'b_figher_total_str': '14 of 17'}
===================536=====================
{'

ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [39]:
import pandas as pd

df = pd.DataFrame(fight_info_list)
df.to_csv("ufc_data.csv", index=False)